First install the packages needed to create the simulation:

In [1]:
install.packages('igraph')
install.packages('animation')
library(igraph)
library(animation)

package 'igraph' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\jamie\AppData\Local\Temp\RtmpKIZzgJ\downloaded_packages
package 'animation' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\jamie\AppData\Local\Temp\RtmpKIZzgJ\downloaded_packages


Warning message:
"package 'igraph' was built under R version 3.6.3"
Attaching package: 'igraph'

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union

Warning message:
"package 'animation' was built under R version 3.6.3"

This program requires having a program called GraphicsMagick installed on your computer. Once you have it installed, tell R the file pathway to GraphicsMagick on your computer:

ani.options("convert"="path-to-file") 

(replace "path-to-file" with the actual file pathway.)

In [2]:
ani.options("convert"="C:/Users/jamie/Downloads/GraphicsMagick-1.3.35-Q16-win64-dll") #pathway on my computer

Now for the simulation code:

In [3]:
# Creates a GIF of a disease spreading through a population according to the Reed-Frost SIR Model
# size = population size
# I0 = initial number of infected individuals
# p = probability of contact between each infected and susceptible person at each time step
# note: the nodes' relative positions on the graph are meaningless and do not affect probability of contact
rfanimation <- function(size, I0, p) {
  g1 <- graph(edges=c(), n=size, directed=F) #create a graph with size vertices, no edges
  Status <- c() #vector representing the status of each person in the population
  for (i in 1:(I0)) { #set the first I0 people to "infected"
    Status <- cbind(Status, "I")
  }
  for (i in 1:(size - I0)) { #set the rest of the population to "susceptible"
    Status <- cbind(Status, "S")
  }
  V(g1)$status = Status # assign the elements in the "Status" vector to the nodes in the graph
  V(g1)$color = ifelse(V(g1)$status=="S", "green", ifelse(V(g1)$status=="I", "red", "gray")) #assign colors
  l <-layout.graphopt(g1) # set the layout of the graph to "graphopt"
  
  #begin the animation
  saveGIF({
    #plot the initial graph
    ## Susceptible "S" = green
    ## Infected "I" = red
    ## Removed "R" = gray
    plot(g1, vertex.color=c("green", "red", "gray")[1+(V(g1)$status=="I")+(2*(V(g1)$status=="R"))], 
         vertex.size=6, vertex.label=NA, layout=l, 
         main=paste("Reed-Frost Simulation, population size =", size, ", p =", p))
    legend(1, 1, legend=c("S", "I", "R"), pch=19, col=c("green", "red", "gray"))
      
    newStatus <- Status #vector for the status of the nodes at the new time step
    
    while ("I" %in% Status) { #while there are infected people in the population  
      for (i in 1:length(Status)) { #for each node in the population
        if (Status[i] == "I") { #if they are infected
          newStatus[i] = "R" #set their new status to recovered          
          for (j in 1:length(Status)) { #traverse the population again
            if (Status[j] == "S") { #if they are susceptible
              u <- runif(1) #create a U ~ Unif(0, 1) random variable
              if (u < p) { #if U is less than the p specified in the parameters
                g1 <- g1 + edge(i,j) #draw an edge between the infected node and the susceptible node
                if (newStatus[j] != "I") { #if node j hasn't already been infected this time step
                  newStatus[j] = "I" #set its new status to infected
                }
              }
            }
          }
        }
      }
  
      Status <- newStatus #set status to new status
      V(g1)$status = Status #assign the new status to each node in the graph
      
      # plot the new time step
      plot(g1, vertex.color=c("green", "red", "gray")[1+(V(g1)$status=="I")+(2*(V(g1)$status=="R"))], 
       vertex.size=6, vertex.label=NA, layout=l,
       main=paste("Reed-Frost Simulation, population size =", size, ", p =", p))
      legend(1, 1, legend=c("S", "I", "R"), pch=19, col=c("green", "red", "gray"))
    }
  
  }, interval=1, movie.name="rf_animation.gif")
}

Finally, we can run the simulation. For example:

In [4]:
rfanimation(150, 1, 0.015)

Output at: rf_animation.gif


[1] TRUE